In [1]:
import xarray as xr
import numpy as np
import glob
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import netCDF4
import pandas as pd
import xesmf as xe
from scipy import signal

In [2]:
var = "psl"
arr = []
for ens_index in range(1,81):
      for ly in range(1,11):
        ly_string = "ly"+str(ly).zfill(2)
        filename = "/work/uo1075/u241321/data/u241321/data_cdo/psl_DJF_80/"+var+"_Amon_MPI-ESM-LR_dkfen41960_2018_r"+str(ens_index)+"i2p1_ym_"+str(ly_string)+"_g1d0_80.nc"
        arr.append(xr.open_dataset(filename,decode_times=False).assign_coords(ensemble=ens_index,ly=ly))

In [3]:
arr

[<xarray.Dataset>
 Dimensions:    (time: 59, bnds: 2, lon: 360, lat: 180)
 Coordinates:
   * time       (time) float64 2.0 14.0 26.0 38.0 ... 662.0 674.0 686.0 698.0
   * lon        (lon) float64 -179.5 -178.5 -177.5 -176.5 ... 177.5 178.5 179.5
   * lat        (lat) float64 -89.5 -88.5 -87.5 -86.5 ... 86.5 87.5 88.5 89.5
     ensemble   int64 1
     ly         int64 1
 Dimensions without coordinates: bnds
 Data variables:
     time_bnds  (time, bnds) float64 ...
     psl        (time, lat, lon) float32 ...
 Attributes:
     CDI:          Climate Data Interface version 2.2.4 (https://mpimet.mpg.de...
     Conventions:  CF-1.6
     institution:  Max Planck Institute for Meteorology
     history:      Thu Dec 07 17:05:58 2023: cdo -f nc4 -z zip_1 -remapbil,glo...
     CDO:          Climate Data Operators version 2.2.2 (https://mpimet.mpg.de...,
 <xarray.Dataset>
 Dimensions:    (time: 59, bnds: 2, lon: 360, lat: 180)
 Coordinates:
   * time       (time) float64 14.0 26.0 38.0 50.0 ... 67

In [6]:
data = xr.concat(arr, dim="ensemble")

In [7]:
ds = data ['psl']

In [8]:
# A reshaped DataArray. In the example in the function comment it would correspond to an array like

#In[1] Ar.dims
#Out[1]: ('year', 'month', 'lat', 'lon')

def xr_reshape(A, dim, newdims, coords):
    """ Reshape DataArray A to convert its dimension dim into sub-dimensions given by
    newdims and the corresponding coords.
    Example: Ar = xr_reshape(A, 'time', ['year', 'month'], [(2017, 2018), np.arange(12)]) """


    # Create a pandas MultiIndex from these labels
    ind = pd.MultiIndex.from_product(coords, names=newdims)

    # Replace the time index in the DataArray by this new index,
    A1 = A.copy()

    A1.coords[dim] = ind

    # Convert multiindex to individual dims using DataArray.unstack().
    # This changes dimension order! The new dimensions are at the end.
    A1 = A1.unstack(dim)

    # Permute to restore dimensions
    i = A.dims.index(dim)
    dims = list(A1.dims)

    for d in newdims[::-1]:
        dims.insert(i, d)

    for d in newdims:
        _ = dims.pop(-1)


    return A1.transpose(*dims)




In [9]:
ds_r = xr_reshape(ds, 'ensemble', ['member', 'lead_year'], [np.arange(1,81),np.arange(1,11),])

In [10]:
annual = ds_r
annual_pre = np.zeros((ds_r['member'].size,ds_r['lead_year'].size,50,ds_r['lat'].size,ds_r['lon'].size))
annual_pre = annual[:,:,9:59,:,:]

annual_prediction = annual_pre.transpose("time", "member", "lead_year", "lat", "lon") # 
dat = np.mean(annual_prediction,axis=0)
dif = annual_prediction-dat

In [14]:
dropped = annual_prediction.stack(feature=("member","lead_year","lat","lon")).dropna(dim="feature")
detrend = signal.detrend(dropped ,axis=0)
feature = dropped .coords["feature"]
time = dropped .coords["time"]
detrend = xr.DataArray(detrend, dims = ["time","feature"], coords = {"time":time,"feature":feature}).unstack()
detrend.to_netcdf("/work/uo1075/u241321/data/psl_DJF_ly_1970-2019_hindcast80_dt.nc")

In [8]:
dif.to_netcdf("/work/uo1075/u241321/data/psl_DJF_ly_1970-2019_hindcast80_new.nc")